In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
sys.path.append("/home/docker_current/py_files/MLCLIP_exp")
sys.path.append("/home/docker_current/py_files/finetune_exp/metaclip_arch")
sys.path.append("/home/docker_current/py_files/utils")
from MLCLIP_utils import get_text_encode_model, get_image_encode_model

device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [2]:
from MLCLIP import MLCLIPPredictor
predictor = MLCLIPPredictor()

# metaclip_model = MetaCLIP(ratio=0.5)
# path_weights = "./weights/ruclip_v5.pth"

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df_test = pd.read_csv("/home/docker_current/datasets/test.csv")
df_test['text_features'] = None
df_test['object_img'] = None

# считываем тестовые изображения, делаем словарь с эмбеддингами
test_images = [i.split('.png')[0] for i in os.listdir("/home/docker_current/datasets/train")]
test_embed = {i:None for i in test_images}

In [4]:
from tqdm.notebook import tqdm

for ind_text in tqdm(range(len(df_test))):
    text = df_test['description'][ind_text]
    text_features = predictor.inference_text(text)
    text_features = torch.Tensor(text_features)
    # with torch.no_grad():
    #     text_features = metaclip_model.encode_image(text_features.to(device)).cpu().detach().numpy()
    df_test['text_features'][ind_text] = text_features

  0%|          | 0/900 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [5]:
from PIL import Image

for name_image in tqdm(test_embed.keys()):
    sample_image_path = "/home/docker_current/datasets/train/" + str(name_image) + ".png"
    image = Image.open(sample_image_path)
    image_features = predictor.inference_image(image)
    image_features = torch.Tensor(image_features)
    # with torch.no_grad():
    #     image_features = metaclip_model.encode_text(image_features.to(device)).cpu().detach().numpy()
    test_embed[name_image] = image_features

  0%|          | 0/2098 [00:00<?, ?it/s]

In [12]:
from copy import deepcopy
copy_test_embed = deepcopy(test_embed)

In [13]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

def get_similarity(image_emb, text_emb):
    sim =cos(torch.Tensor(image_emb), 
                        torch.Tensor(text_emb))
    return sim

In [15]:
preds = []

for ind_text in tqdm(range(len(df_test))):

    sims = []
    for image_name in copy_test_embed.keys():
        sim = get_similarity(copy_test_embed[image_name], 
                                    df_test['text_features'][ind_text])
        sims.append(sim)

    sims = np.array(sims)
    ind_max = np.argmax(sims)
    match_image = list(copy_test_embed.keys())[ind_max]
    preds.append(match_image)
    # del copy_test_embed[match_image]

  0%|          | 0/900 [00:00<?, ?it/s]

In [17]:
submit = pd.read_csv('/home/docker_current/py_files/sample_solution.csv')
submit['object_img'] = preds
submit['object_img'] = submit['object_img'].astype(np.int64)
submit.to_csv('./submits/dummy_2_repeat.csv', index=False)

In [16]:
len(copy_test_embed.keys())

2098